In [1]:
import json 
from pprint import pprint

import pandas as pd

import numpy as np
from pandas.io.json import json_normalize

### Converting Json --> Dataframe 

In [2]:
def extractDataInfo(filename):
    data = []
    attributes = []
    returnValues = []
    with open(filename) as f:
        for line in f:
            dataLine = json.loads(line)
            data.append(dataLine)
            for key in dataLine.keys():
                attributes.append(key)
        uniqueAttributes = set(attributes)
        returnValues.append(data)
        returnValues.append(uniqueAttributes)
        return returnValues

In [3]:
businessInfo = extractDataInfo('../dataset/business.json') #businessInfo is a list

In [4]:
businessDf = pd.DataFrame.from_dict(businessInfo[0])

In [5]:
originalDf = pd.DataFrame.from_dict(businessInfo[0]) #the original dataset 

### Summary of Raw Dataframe 

In [5]:
print(businessInfo[0][0])

{'business_id': 'YDf95gJZaq05wvo7hTQbbQ', 'name': 'Richmond Town Square', 'neighborhood': '', 'address': '691 Richmond Rd', 'city': 'Richmond Heights', 'state': 'OH', 'postal_code': '44143', 'latitude': 41.5417162, 'longitude': -81.4931165, 'stars': 2.0, 'review_count': 17, 'is_open': 1, 'attributes': {'RestaurantsPriceRange2': 2, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'BikeParking': True, 'WheelchairAccessible': True}, 'categories': ['Shopping', 'Shopping Centers'], 'hours': {'Monday': '10:00-21:00', 'Tuesday': '10:00-21:00', 'Friday': '10:00-21:00', 'Wednesday': '10:00-21:00', 'Thursday': '10:00-21:00', 'Sunday': '11:00-18:00', 'Saturday': '10:00-21:00'}}


In [6]:
businessDf.shape

(156639, 15)

Part II: Describe the Data 
The data type of each variable

In [7]:
businessDf.dtypes

address          object
attributes       object
business_id      object
categories       object
city             object
hours            object
is_open           int64
latitude        float64
longitude       float64
name             object
neighborhood     object
postal_code      object
review_count      int64
stars           float64
state            object
dtype: object

A brief description of what the variable contains (i.e. "The Tuition column contains information on the annual cost of tuition in $USD")
"business_id":string, 22 character unique string business id
"name":string, the business's name
"neighborhood": string, the neighborhood's name
"address": string, the full address of the business
"city": string, the city
"State": string, 2 character state code, if applicable
"postal code": string, the postal code
"latitude": float, latitude
"longitude": float, longitude
"stars": float, star rating, rounded to half-stars
"review_count": interger, number of reviews
"is_open": integer, 0 or 1 for closed or open, respectively
"attributes": object, business attributes to values. note: some attribute values might be objects
"categories": an array of strings of business categories
"hours":an object of key day to value hours, hours are using a 24hr clock
More description at https://www.yelp.com/dataset/documentation/json

Summarize the values as appropriate. Look at the min, max, and missing values. Do they make sense?

In [8]:
businessDf.describe()

,is_open,latitude,longitude,review_count,stars
count,156639.000000,156638.000000,156638.000000,156639.000000,156639.000000
mean,0.844375,38.585033,-92.856485,30.238159,3.647154
std,0.362501,5.399871,26.557741,96.486631,0.977640
min,0.000000,-36.086009,-142.466650,3.000000,1.000000
25%,1.000000,33.627161,-112.138207,4.000000,3.000000
50%,1.000000,36.142381,-89.523198,9.000000,3.500000
75%,1.000000,43.596845,-79.668760,23.000000,4.500000
max,1.000000,89.999314,115.086769,6979.000000,5.000000


### Unnesting Jsons --> Expanded DataFrame 

In [6]:
def unnestJson(dataframe):
    data = list(dataframe)
    return json_normalize(data)

In [7]:
openHoursDf = unnestJson(businessDf['hours']) #The dataframe with unnested open hours info 


In [8]:
attributesDf = unnestJson(businessDf['attributes']) #The dataframe with unnested attributes info 

### Looked At Missing Data to Clean / Condense dataset 

In [23]:
 def missingDataSummary(df):
    #dfInitial = pd.DataFrame(index = [df.name], columns =df.columns)
    dfInitial = df.iloc[0]
    for column in df.columns:
        #print("Column name: " + column )
        numNas = df[column].isnull().sum()
        numObservations = len(df[column])
        missingDataPercentage = numNas/numObservations *100
        dfInitial[column] = missingDataPercentage
        #print(str(missingDataPercentage) + "% missing") 
    return dfInitial

In [9]:
pd.set_option('display.max_columns', None)


In [11]:
missingAttributesDf = missingDataSummary(businessDf)
print(missingAttributesDf )

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


address                   0
attributes                0
business_id               0
categories                0
city                      0
hours                     0
is_open                   0
latitude        0.000638411
longitude       0.000638411
name                      0
neighborhood              0
postal_code               0
review_count              0
stars                     0
state                     0
Name: 0, dtype: object


### Combine Unnested Json Dataframes to --> businessDf

In [10]:
businessDf = pd.concat([businessDf, attributesDf], axis = 1, join = 'inner')

In [11]:
#merge two dataframes sidebyside by columns 
businessDf = pd.concat([businessDf, openHoursDf], axis=1, join='inner') 

In [ ]:
#find relationship between stars and review count perhaps 

### Figure out the counts by region -- To group by region & Delete insiginificant regions 

In [12]:
uniqueStates = businessDf['state'].unique()
statesCount = {}
for state in businessDf['state']:
    if(state in statesCount):
        statesCount[state] = statesCount[state]+1 
    else:
        statesCount[state] = 1; 
     

In [15]:
statesCount

{'01': 6,
 '3': 1,
 '75': 1,
 'ABE': 2,
 'AL': 2,
 'AZ': 47376,
 'BW': 3071,
 'BY': 4,
 'C': 32,
 'CA': 6,
 'DE': 1,
 'EDH': 3561,
 'ELN': 42,
 'ESX': 11,
 'FAL': 1,
 'FIF': 73,
 'FL': 1,
 'FLN': 1,
 'GLG': 1,
 'HH': 1,
 'HLD': 175,
 'IL': 1667,
 'KHL': 1,
 'MLN': 185,
 'NC': 11299,
 'NE': 1,
 'NI': 11,
 'NLK': 1,
 'NTH': 2,
 'NV': 30571,
 'NY': 15,
 'NYK': 42,
 'OH': 10930,
 'ON': 26520,
 'PA': 8916,
 'PKN': 1,
 'QC': 7273,
 'RCC': 1,
 'SC': 583,
 'SCB': 5,
 'SL': 1,
 'ST': 9,
 'STG': 1,
 'TAM': 1,
 'VT': 1,
 'WA': 3,
 'WHT': 1,
 'WI': 4190,
 'WLN': 36,
 'XGL': 2,
 'ZET': 1}

In [13]:
importantStatesCount = statesCount.copy()
for mapping in statesCount:
    if(importantStatesCount[mapping] < 500):
        del importantStatesCount[mapping]


In [14]:
importantStatesCount #will combine NC with SC 

{'AZ': 47376,
 'BW': 3071,
 'EDH': 3561,
 'IL': 1667,
 'NC': 11299,
 'NV': 30571,
 'OH': 10930,
 'ON': 26520,
 'PA': 8916,
 'QC': 7273,
 'SC': 583,
 'WI': 4190}

In [16]:
statesList = list(importantStatesCount.keys())

In [17]:
statesList

['OH', 'NC', 'ON', 'AZ', 'PA', 'NV', 'EDH', 'QC', 'WI', 'IL', 'BW', 'SC']

In [18]:
subsetBusinessDf = businessDf[businessDf['state'].isin(statesList)]

In [26]:
len(businessDf)

156639

In [27]:
len(subsetBusinessDf)

155957

In [19]:
subsetBusinessDf #The new cleaned dataset with delted regions 

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,AcceptsInsurance,AgesAllowed,Alcohol,Ambience.casual,Ambience.classy,Ambience.divey,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale,BYOB,BYOBCorkage,BestNights.friday,BestNights.monday,BestNights.saturday,BestNights.sunday,BestNights.thursday,BestNights.tuesday,BestNights.wednesday,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet,BusinessParking.validated,ByAppointmentOnly,Caters,CoatCheck,Corkage,DietaryRestrictions.dairy-free,DietaryRestrictions.gluten-free,DietaryRestrictions.halal,DietaryRestrictions.kosher,DietaryRestrictions.soy-free,DietaryRestrictions.vegan,DietaryRestrictions.vegetarian,DogsAllowed,DriveThru,GoodForDancing,GoodForKids,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch,HairSpecializesIn.africanamerican,HairSpecializesIn.asian,HairSpecializesIn.coloring,HairSpecializesIn.curly,HairSpecializesIn.extensions,HairSpecializesIn.kids,HairSpecializesIn.perms,HairSpecializesIn.straightperms,HappyHour,HasTV,Music.background_music,Music.dj,Music.jukebox,Music.karaoke,Music.live,Music.no_music,Music.video,NoiseLevel,Open24Hours,OutdoorSeating,RestaurantsAttire,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,False,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,True,NaN,10:00-21:00,10:00-21:00,10:00-21:00,11:00-18:00,10:00-21:00,10:00-21:00,10:00-21:00
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,average,NaN,False,casual,NaN,True,True,2.0,False,NaN,True,NaN,NaN,NaN,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,False,False,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,True,no,10:00-19:00,10:00-19:00,10:00-18:00,12:00-17:00,10:00-19:00,10:00-19:00,10:00-19:00
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,

### Summarize Missing Info data of new dataset & compare with original 

In [22]:
missingDataSummary(originalDf)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


address                   0
attributes                0
business_id               0
categories                0
city                      0
hours                     0
is_open                   0
latitude        0.000638411
longitude       0.000638411
name                      0
neighborhood              0
postal_code               0
review_count              0
stars                     0
state                     0
Name: 0, dtype: object

In [24]:
missingSubsetInfo = missingDataSummary(subsetBusinessDf)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
missingSubsetInfo

address                                0
attributes                             0
business_id                            0
categories                             0
city                                   0
hours                                  0
is_open                                0
latitude                     0.000641202
longitude                    0.000641202
name                                   0
neighborhood                           0
postal_code                            0
review_count                           0
stars                                  0
state                                  0
AcceptsInsurance                 94.5036
AgesAllowed                      99.7467
Alcohol                          71.7717
Ambience.casual                  72.6046
Ambience.classy                  72.6046
Ambience.divey                   82.5772
Ambience.hipster                 72.6046
Ambience.intimate                72.6046
Ambience.romantic                72.6046
Ambience.tourist

In [25]:
subsetMissingDf = pd.DataFrame(missingSubsetInfo)

In [26]:
subsetMissingDf = subsetMissingDf.T #to switch rows to columns 
subsetMissingDf

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,AcceptsInsurance,AgesAllowed,Alcohol,Ambience.casual,Ambience.classy,Ambience.divey,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale,BYOB,BYOBCorkage,BestNights.friday,BestNights.monday,BestNights.saturday,BestNights.sunday,BestNights.thursday,BestNights.tuesday,BestNights.wednesday,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet,BusinessParking.validated,ByAppointmentOnly,Caters,CoatCheck,Corkage,DietaryRestrictions.dairy-free,DietaryRestrictions.gluten-free,DietaryRestrictions.halal,DietaryRestrictions.kosher,DietaryRestrictions.soy-free,DietaryRestrictions.vegan,DietaryRestrictions.vegetarian,DogsAllowed,DriveThru,GoodForDancing,GoodForKids,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch,HairSpecializesIn.africanamerican,HairSpecializesIn.asian,HairSpecializesIn.coloring,HairSpecializesIn.curly,HairSpecializesIn.extensions,HairSpecializesIn.kids,HairSpecializesIn.perms,HairSpecializesIn.straightperms,HappyHour,HasTV,Music.background_music,Music.dj,Music.jukebox,Music.karaoke,Music.live,Music.no_music,Music.video,NoiseLevel,Open24Hours,OutdoorSeating,RestaurantsAttire,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,0,0,0,0,0.000641202,0.000641202,0,0,0,0,0,0,94.5036,99.7467,71.7717,72.6046,72.6046,82.5772,72.6046,72.6046,72.6046,72.6046,72.6046,72.6046,99.4197,99.1023,95.9149,95.9149,95.9149,95.9149,95.9149,95.9149,95.9149,53.0024,94.3157,22.8313,42.8336,42.8336,42.8336,42.8336,44.1102,77.7131,77.8997,94.9172,99.5819,99.8435,99.8435,99.8435,99.8435,99.8435,99.8435,99.8435,92.832,96.1355,94.5632,62.7282,72.563,72.563,72.563,72.563,72.563,72.563,99.2472,99.2472,99.0998,99.0998,99.0998,99.0998,99.0998,99.2472,94.6351,72.091,94.8838,94.8838,94.8838,94.8838,94.8838,94.8838,94.8838,74.0377,99.7775,67.8931,71.387,99.7461,70.1129,68.356,38.5113,70.3104,74.4648,65.1032,95.1044,71.8557,72.268,27.4916,32.5571,35.3155,51.9688,27.2158,28.2193,27.5076


In [27]:
len(subsetMissingDf.iloc[0])

103

 ### Delete Attribute columns that have more than 90% of its data missing 

In [28]:
columnIndex = 0; 
delCols = []
for column in subsetMissingDf.columns:
    if(subsetMissingDf[column][0] > 90):
        print('Greater than 90!')
        print(column )
        delCols.append(column)
        print(columnIndex)
    columnIndex = columnIndex + 1

    #print(column)
    #print(type(column))
    #print(subsetMissingDf[column][0])

Greater than 90!
AcceptsInsurance
15
Greater than 90!
AgesAllowed
16
Greater than 90!
BYOB
27
Greater than 90!
BYOBCorkage
28
Greater than 90!
BestNights.friday
29
Greater than 90!
BestNights.monday
30
Greater than 90!
BestNights.saturday
31
Greater than 90!
BestNights.sunday
32
Greater than 90!
BestNights.thursday
33
Greater than 90!
BestNights.tuesday
34
Greater than 90!
BestNights.wednesday
35
Greater than 90!
BusinessAcceptsBitcoin
37
Greater than 90!
CoatCheck
46
Greater than 90!
Corkage
47
Greater than 90!
DietaryRestrictions.dairy-free
48
Greater than 90!
DietaryRestrictions.gluten-free
49
Greater than 90!
DietaryRestrictions.halal
50
Greater than 90!
DietaryRestrictions.kosher
51
Greater than 90!
DietaryRestrictions.soy-free
52
Greater than 90!
DietaryRestrictions.vegan
53
Greater than 90!
DietaryRestrictions.vegetarian
54
Greater than 90!
DogsAllowed
55
Greater than 90!
DriveThru
56
Greater than 90!
GoodForDancing
57
Greater than 90!
HairSpecializesIn.africanamerican
65
Greate

In [29]:
delCols

['AcceptsInsurance',
 'AgesAllowed',
 'BYOB',
 'BYOBCorkage',
 'BestNights.friday',
 'BestNights.monday',
 'BestNights.saturday',
 'BestNights.sunday',
 'BestNights.thursday',
 'BestNights.tuesday',
 'BestNights.wednesday',
 'BusinessAcceptsBitcoin',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions.dairy-free',
 'DietaryRestrictions.gluten-free',
 'DietaryRestrictions.halal',
 'DietaryRestrictions.kosher',
 'DietaryRestrictions.soy-free',
 'DietaryRestrictions.vegan',
 'DietaryRestrictions.vegetarian',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'HairSpecializesIn.africanamerican',
 'HairSpecializesIn.asian',
 'HairSpecializesIn.coloring',
 'HairSpecializesIn.curly',
 'HairSpecializesIn.extensions',
 'HairSpecializesIn.kids',
 'HairSpecializesIn.perms',
 'HairSpecializesIn.straightperms',
 'HappyHour',
 'Music.background_music',
 'Music.dj',
 'Music.jukebox',
 'Music.karaoke',
 'Music.live',
 'Music.no_music',
 'Music.video',
 'Open24Hours',
 'RestaurantsCounterService',
 'Smoking

In [30]:
newSubsetDf = subsetMissingDf.drop( delCols, axis=1) # clean 90% or more missing data

In [31]:
len(newSubsetDf.iloc[0])

60

In [32]:
newSubsetDf

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,Alcohol,Ambience.casual,Ambience.classy,Ambience.divey,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet,BusinessParking.validated,ByAppointmentOnly,Caters,GoodForKids,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,0,0,0,0,0.000641202,0.000641202,0,0,0,0,0,0,71.7717,72.6046,72.6046,82.5772,72.6046,72.6046,72.6046,72.6046,72.6046,72.6046,53.0024,22.8313,42.8336,42.8336,42.8336,42.8336,44.1102,77.7131,77.8997,62.7282,72.563,72.563,72.563,72.563,72.563,72.563,72.091,74.0377,67.8931,71.387,70.1129,68.356,38.5113,70.3104,74.4648,65.1032,71.8557,72.268,27.4916,32.5571,35.3155,51.9688,27.2158,28.2193,27.5076


In [98]:
len(businessDf)

156639

In [33]:
businessDf =  businessDf.drop( delCols, axis=1) # clean 90% or more missing data

In [34]:
len(businessDf)

156639

In [35]:
businessDf.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,Alcohol,Ambience.casual,Ambience.classy,Ambience.divey,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet,BusinessParking.validated,ByAppointmentOnly,Caters,GoodForKids,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,True,NaN,10:00-21:00,10:00-21:00,10:00-21:00,11:00-18:00,10:00-21:00,10:00-21:00,10:00-21:00
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC,NaN,False,False,False,False,False,False,False,False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,average,False,casual,True,True,2.0,False,NaN,True,NaN,NaN,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,False,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,2.0,NaN,NaN,NaN,True,no,10:00-19:00,10:00-19:00,10:00-18:00,12:00-17:00,10:00-19:00,10:00-19:00,10:00-19:00
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,3.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:00-17:00,9:00-17:00,NaN,NaN,9:00-17:00,9:00-17:00,9:00-17:00
4,4719 N 20Th St,"{'RestaurantsTableService': False, 'GoodForMea...",duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]",Phoenix,{},0,33.505928,-112.038847,Blimpie,,85016,10,4.5,AZ,none,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,NaN,True,True,False,False,False,False,False,False,False,quiet,False,casual,False,True,1.0,False,False,True,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Checking if the ratios & dataset has any overlapping columns to delete

In [36]:
dataRatio = {}
for column in newSubsetDf.columns: 
    if newSubsetDf[column][0] in dataRatio:
        dataRatio[newSubsetDf[column][0]].append(column)
    else: 
        dataRatio[newSubsetDf[column][0]] = [column]

In [37]:
dataRatio

{0.0: ['address',
  'attributes',
  'business_id',
  'categories',
  'city',
  'hours',
  'is_open',
  'name',
  'neighborhood',
  'postal_code',
  'review_count',
  'stars',
  'state'],
 0.00064120238270805416: ['latitude', 'longitude'],
 22.831293241085685: ['BusinessAcceptsCreditCards'],
 27.21583513404336: ['Thursday'],
 27.491552158607824: ['Friday'],
 27.507582218175525: ['Wednesday'],
 28.219316862981465: ['Tuesday'],
 32.557050982001449: ['Monday'],
 35.315503632411499: ['Saturday'],
 38.51125630782844: ['RestaurantsPriceRange2'],
 42.833601569663429: ['BusinessParking.garage',
  'BusinessParking.lot',
  'BusinessParking.street',
  'BusinessParking.valet'],
 44.110235513635168: ['BusinessParking.validated'],
 51.968811916105082: ['Sunday'],
 53.002430157030467: ['BikeParking'],
 62.728187897946228: ['GoodForKids'],
 65.10320152349685: ['RestaurantsTakeOut'],
 67.893073090659612: ['OutdoorSeating'],
 68.356021210974816: ['RestaurantsGoodForGroups'],
 70.112915739594882: ['Restau

In [41]:
parkingAttr = dataRatio[42.833601569663429]

In [42]:
mealAttr = dataRatio[72.562950043922356]

In [53]:
ambienceAttr = dataRatio[72.60462819879838]

In [49]:
parkingDf = businessDf[parkingAttr]

parkingDf

,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet
0,False,True,False,False
1,NaN,NaN,NaN,NaN
2,False,False,True,False
3,NaN,NaN,NaN,NaN
4,False,False,False,False
5,False,True,False,False
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,False,True,False,False
9,NaN,NaN,NaN,NaN


In [50]:
mealDf = businessDf[mealAttr]

mealDf

,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch
0,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,False,False,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,False,False,False
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
ambienceDf = businessDf[ambienceAttr]

ambienceDf

,Ambience.casual,Ambience.classy,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,False,False,False,False,False,False
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,False,False,False,False,False
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dividing Dataframe by Region 

In [57]:
businessDf.loc[businessDf['state'] == 'AZ']

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,Alcohol,Ambience.casual,Ambience.classy,Ambience.divey,Ambience.hipster,Ambience.intimate,Ambience.romantic,Ambience.touristy,Ambience.trendy,Ambience.upscale,BikeParking,BusinessAcceptsCreditCards,BusinessParking.garage,BusinessParking.lot,BusinessParking.street,BusinessParking.valet,BusinessParking.validated,ByAppointmentOnly,Caters,GoodForKids,GoodForMeal.breakfast,GoodForMeal.brunch,GoodForMeal.dessert,GoodForMeal.dinner,GoodForMeal.latenight,GoodForMeal.lunch,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,3.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:00-17:00,9:00-17:00,NaN,NaN,9:00-17:00,9:00-17:00,9:00-17:00
4,4719 N 20Th St,"{'RestaurantsTableService': False, 'GoodForMea...",duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]",Phoenix,{},0,33.505928,-112.038847,Blimpie,,85016,10,4.5,AZ,none,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,NaN,True,True,False,False,False,False,False,False,False,quiet,False,casual,False,True,1.0,False,False,True,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017 E Camelback Rd,"{'BusinessAcceptsCreditCards': True, 'Restaura...",uUEMrhJiL1a1pCA_I1SU7Q,"[Shopping, Tobacco Shops]",Phoenix,"{'Monday': '10:00-20:00', 'Tuesday': '10:00-20...",0,33.508068,-112.037552,Baxter's Cigars,,85016,35,5.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,10:00-21:00,10:00-20:00,10:00-21:00,10:00-19:00,10:00-20:00,10:00-20:00,10:00-20:00
6,"4425 N 24th St, Ste 125","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",2eJEUJIP54tex7T9YOcLSw,"[Chiropractors, Health & Medical]",Phoenix,"{'Friday': '9:00-12:00', 'Tuesday': '14:00-19:...",1,33.502848,-112.012696,Back-Health Chiropractic,,85016,19,5.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:00-12:00,14:30-17:00,NaN,NaN,14:00-19:00,14:00-19:00,14:30-17:00
9,"1425 S Higley Rd, Ste 103","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",NqiQdFa93wzUJGo29NbTPQ,"[Health & Medical, Optometrists]",Gilbert,"{'Friday': '7:30-11:00', 'Tuesday': '7:30-17:0...",1,33.324539,-111.720449,Neighborhood Vision Center,,85296,8,5.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:30-11:00,7:30-17:00,NaN,NaN,7:30-17:00,7:30-17:00,7:30-17:00
11,"9393 N 90th St, Ste 112","{'BusinessParking': {'garage': False, 'street'...",6s3z3TlpHOIecuSyPEOp7A,"[Barbers, Men's Hair Salons, Hair Removal, Wax...",Scottsdale,"{'Monday': '10:00-18:00', 'Tuesday': '10:00-18...",0,33.570421,-111.886492,Sq Cutz,,85258,7,4.5,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,True,NaN,9:00-19:00,10:00-18:00,9:00-18:00,NaN,10:00-18:00,10:00-18:00,10:00-18:00
12,9101 E Baseline Rd,"{'BusinessParking': {'garage': False, 'street'...",n33Izvzk_z9_51H6NsQF-A,"[Flowers & Gifts, Bakeries, Grocery, Shopping,...",Mesa,"{'Monday': '5:00-0:00', 'Tuesday': '5:00-0:00'...",1,33.377428,-111.638108,Safeway,,85209,21,2.0,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,Fal

In [100]:
businessDf = businessDf.drop(['address', 'attributes', 'hours'], axis = 1)

In [101]:
ohDf = businessDf.loc[businessDf['state'] == 'OH']
ncDf = businessDf.loc[businessDf['state'] == 'NC']
onDf = businessDf.loc[businessDf['state'] == 'ON']
azDf = businessDf.loc[businessDf['state'] == 'AZ']
paDf = businessDf.loc[businessDf['state'] == 'PA']
nvDf = businessDf.loc[businessDf['state'] == 'NV']
edhDf = businessDf.loc[businessDf['state'] == 'EDH']
qcDf = businessDf.loc[businessDf['state'] == 'QC']
wiDf = businessDf.loc[businessDf['state'] == 'WI']
ilDf = businessDf.loc[businessDf['state'] == 'IL']
bwDf = businessDf.loc[businessDf['state'] == 'BW']
scDf = businessDf.loc[businessDf['state'] == 'SC']

In [122]:
carolinaDf = ncDf.append(scDf)
centralDf = wiDf.append(ilDf)

In [123]:
statesDataFrames = [ohDf, carolinaDf, onDf, azDf, paDf, nvDf, edhDf, qcDf, centralDf, bwDf]

In [ ]:
for statesDf in statesDataFrames:
    statesDf = statesDf.reset_index()
    print("This state has..." + statesDf['state'].unique())
    print(len(statesDf))
    

In [ ]:
missingDataSummary(ohDf)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
